**Safe Deployment: Deploy and score a machine learning model after testing the online endpoint**

Test the deployment with Mirrored traffic
Test the deployment with some percentage of live traffic
once satisfied with the deployment, assign 100% live traffic a

Workflow 

1. Initialize Workspace & create workspace handle 
2. Initialize Environment 
3. configure Endpoint 
4. Get reference of the register model 
5. create a scoring script 
6. Configure Deployment 
7. Assign traffic to the deployment 
8. Test the Deployment 
9. Create new deployment with New or enhanced model 
10. Test the Deployment with Mirrored traffic. After Testing set the mirrored traffic to 0 
11. Send some percentage of live traffic to new deployment 
12. Rollout New Deployment 
13. Remove Old Deployment 
14. Delete the Endpoint



##### Step 1: Initialize Workspace and Create Workspace handle

In [5]:
from azureml.core import Workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Initialize  workspace
ws = Workspace.from_config()  

# Get a handle to the workspace
credential = DefaultAzureCredential()  # authenticate
ml_client = MLClient( credential=credential,
                      subscription_id=ws.subscription_id,
                      resource_group_name=ws.resource_group,
                      workspace_name=ws.name,
                    )


##### Step 2: Initialize Environment

In [6]:
import os
from azure.ai.ml.entities import Environment

custom_env_name  = "ENV-SDKv2"
# dependencies_dir = '../dependencies'
# env = Environment( name=custom_env_name,
#                    description="Evironment for python SDKv2 Execution",
#                    conda_file=os.path.join(dependencies_dir, "conda.yaml"),
#                    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
#                  )
# env = ml_client.environments.create_or_update(env)

# GET ENVIRONMENT
# use 'label' parameter to get latest environment for example label='latest'
# use 'version' parameter to get specific version environment, for example version=2
env = ml_client.environments.get(name=custom_env_name, label='latest') 

print(f"Environment with name {env.name} is registered to workspace, the environment version is {env.version}")

Environment with name ENV-SDKv2 is registered to workspace, the environment version is 6


##### Step 3: Configure Endpoint
Note: auth_mode is either **key** or **token**. The key difference is **token** expires and **key** won't

In [7]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

# online endpoint
online_endpoint_name = "pima-endpoint-SDKv2-v5"

endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint for mlflow model",
    auth_mode="key",
    tags={"SDK": "2"},
)
# create endpoint
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://pima-endpoint-sdkv2-v5.centralindia.inference.ml.azure.com/score', 'openapi_uri': 'https://pima-endpoint-sdkv2-v5.centralindia.inference.ml.azure.com/swagger.json', 'name': 'pima-endpoint-sdkv2-v5', 'description': 'this is a sample online endpoint for mlflow model', 'tags': {'SDK': '2'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/resourcegroups/us_azure_practice/providers/microsoft.machinelearningservices/workspaces/us_azure/onlineendpoints/pima-endpoint-sdkv2-v5', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/providers/Microsoft.MachineLearningServices/locations/centralindia/mfeOperationsStatus/oe:a78ffdcd-763f-4353-b2d5-5456db1b024f:6ef4e53f-06ec-417e-8e29-329caaa8c1ac?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/ba5d6a04-

##### Step 4: Get referrence to Registered Model

In [8]:
# referencing model from model registry
model_name = 'pima_model_SDKv2_02'
model = ml_client.models.get(name=model_name, label="latest")

##### Step 5: Create a Scoring Script and save in 'src' folder

##### Step 6: Configure Deployment

**Instance type**: The VM size to use for the deployment. For the list of supported sizes.[cick here](https://learn.microsoft.com/en-us/azure/machine-learning/reference-managed-online-endpoints-vm-sku-list?view=azureml-api-2)                                                                                                        

In [9]:
from azure.ai.ml.entities import ManagedOnlineDeployment, CodeConfiguration

deployment_name = "blue-deployment"
deployment_blue = ManagedOnlineDeployment(
                name=deployment_name, # deployment name
                endpoint_name=online_endpoint_name,  
                model=model,
                environment=env,
                code_configuration=CodeConfiguration(
                code="../src", scoring_script="pima_scoreOnlineEndpoint_SDKv2.py"
                ),
                instance_type="Standard_F4s_v2",
                instance_count=1,
            )

ml_client.online_deployments.begin_create_or_update(deployment=deployment_blue).result() # create local deployment


Check: endpoint pima-endpoint-SDKv2-v5 exists


...................................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'provisioning_state': 'Succeeded', 'endpoint_name': 'pima-endpoint-sdkv2-v5', 'type': 'Managed', 'name': 'blue-deployment', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/providers/Microsoft.MachineLearningServices/locations/centralindia/mfeOperationsStatus/od:a78ffdcd-763f-4353-b2d5-5456db1b024f:cb65ff8e-0195-4f21-8e89-58e1ca177805?api-version=2023-04-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/resourceGroups/us_azure_practice/providers/Microsoft.MachineLearningServices/workspaces/us_azure/onlineEndpoints/pima-endpoint-sdkv2-v5/deployments/blue-deployment', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/instancenotebook/code/Users/uj_az/PIMA- OnlineEndpoint/02-Deploying-Online-Endpoint/SDKv2/notebooks', 'creation_context':

##### Step 7: Assign traffic to the deployment

In [10]:
# assign traffic to the deployment
endpoint.traffic = {deployment_name: 100}
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://pima-endpoint-sdkv2-v5.centralindia.inference.ml.azure.com/score', 'openapi_uri': 'https://pima-endpoint-sdkv2-v5.centralindia.inference.ml.azure.com/swagger.json', 'name': 'pima-endpoint-sdkv2-v5', 'description': 'this is a sample online endpoint for mlflow model', 'tags': {'SDK': '2'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/resourcegroups/us_azure_practice/providers/microsoft.machinelearningservices/workspaces/us_azure/onlineendpoints/pima-endpoint-sdkv2-v5', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/providers/Microsoft.MachineLearningServices/locations/centralindia/mfeOperationsStatus/oe:a78ffdcd-763f-4353-b2d5-5456db1b024f:424d0ce7-abe9-462b-9c46-767154cae80b?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/ba5d6a04-

##### Step 8: Test the endpoint with sample data
We will send a sample request using a [json](../../data/sample_request.json) file. 

In [11]:
# testing the endpoint
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name=deployment_name,
    request_file="../../data/sample_request.json",
)

'[0, 0]'

##### Step 9: Create New Deployment - green

In [13]:
# Get referrence to you new or enhanced model from model registry
model_name = 'pima_model_SDKv2_02'
model2 = ml_client.models.get(name=model_name, label='latest')


# configure and create you new deployment
deployment_name2 = 'green' 
green_deployment = ManagedOnlineDeployment(
    name=deployment_name2,
    endpoint_name=online_endpoint_name,
    model=model2,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../src", scoring_script="pima_scoreOnlineEndpoint_SDKv2.py"
    ),
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

ml_client.online_deployments.begin_create_or_update(deployment=green_deployment).result() 

Instance type Standard_DS2_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/en-us/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint pima-endpoint-SDKv2-v5 exists


......................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'provisioning_state': 'Succeeded', 'endpoint_name': 'pima-endpoint-sdkv2-v5', 'type': 'Managed', 'name': 'green', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/providers/Microsoft.MachineLearningServices/locations/centralindia/mfeOperationsStatus/od:a78ffdcd-763f-4353-b2d5-5456db1b024f:6fe4c333-9330-4cfd-9a85-91ba697e24bd?api-version=2023-04-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/resourceGroups/us_azure_practice/providers/Microsoft.MachineLearningServices/workspaces/us_azure/onlineEndpoints/pima-endpoint-sdkv2-v5/deployments/green', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/instancenotebook/code/Users/uj_az/PIMA- OnlineEndpoint/02-Deploying-Online-Endpoint/SDKv2/notebooks', 'creation_context': None, 'serialize': 

##### Step 10: Test the deployment with mirrored traffic
Once you've tested your `green` deployment, you can copy (or 'mirror') a percentage of the live traffic to it. Mirroring traffic doesn't change results returned to clients. Requests still flow 100% to the blue deployment. The mirrored percentage of the traffic is copied and submitted to the `green` deployment so you can gather metrics and logging without impacting your clients. Mirroring is useful when you want to validate a new deployment without impacting clients. For example, to check if latency is within acceptable bounds and that there are no HTTP errors.

In [14]:
endpoint.mirror_traffic = {deployment_name2: 30}
ml_client.begin_create_or_update(endpoint).result()


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://pima-endpoint-sdkv2-v5.centralindia.inference.ml.azure.com/score', 'openapi_uri': 'https://pima-endpoint-sdkv2-v5.centralindia.inference.ml.azure.com/swagger.json', 'name': 'pima-endpoint-sdkv2-v5', 'description': 'this is a sample online endpoint for mlflow model', 'tags': {'SDK': '2'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/resourcegroups/us_azure_practice/providers/microsoft.machinelearningservices/workspaces/us_azure/onlineendpoints/pima-endpoint-sdkv2-v5', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/providers/Microsoft.MachineLearningServices/locations/centralindia/mfeOperationsStatus/oe:a78ffdcd-763f-4353-b2d5-5456db1b024f:e87b2529-0d21-46ce-9a45-dec186c44ade?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/ba5d6a04-

In [16]:
# testing the endpoint
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name=deployment_name2,
    request_file="../../data/sample_request.json",
)

'[0, 0]'

In [17]:
# After Testing set the mirror_traffic to 0
endpoint.mirror_traffic = {deployment_name2: 0}
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://pima-endpoint-sdkv2-v5.centralindia.inference.ml.azure.com/score', 'openapi_uri': 'https://pima-endpoint-sdkv2-v5.centralindia.inference.ml.azure.com/swagger.json', 'name': 'pima-endpoint-sdkv2-v5', 'description': 'this is a sample online endpoint for mlflow model', 'tags': {'SDK': '2'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/resourcegroups/us_azure_practice/providers/microsoft.machinelearningservices/workspaces/us_azure/onlineendpoints/pima-endpoint-sdkv2-v5', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/providers/Microsoft.MachineLearningServices/locations/centralindia/mfeOperationsStatus/oe:a78ffdcd-763f-4353-b2d5-5456db1b024f:afc9527f-9b61-4191-9941-c32b3a604c91?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/ba5d6a04-

##### Step 11: Test the new deployment with some percentage of live-traffic

In [18]:
endpoint.traffic = {deployment_name: 90, deployment_name2: 10}
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://pima-endpoint-sdkv2-v5.centralindia.inference.ml.azure.com/score', 'openapi_uri': 'https://pima-endpoint-sdkv2-v5.centralindia.inference.ml.azure.com/swagger.json', 'name': 'pima-endpoint-sdkv2-v5', 'description': 'this is a sample online endpoint for mlflow model', 'tags': {'SDK': '2'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/resourcegroups/us_azure_practice/providers/microsoft.machinelearningservices/workspaces/us_azure/onlineendpoints/pima-endpoint-sdkv2-v5', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/providers/Microsoft.MachineLearningServices/locations/centralindia/mfeOperationsStatus/oe:a78ffdcd-763f-4353-b2d5-5456db1b024f:b91695da-e420-4d30-8922-6c79b70557df?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/ba5d6a04-

##### Step 12: Rollout New Deployment
once you're satisfied with the deployment send all the live-traffic to the new deployment


In [19]:
endpoint.traffic = {deployment_name: 0, deployment_name2: 100}
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://pima-endpoint-sdkv2-v5.centralindia.inference.ml.azure.com/score', 'openapi_uri': 'https://pima-endpoint-sdkv2-v5.centralindia.inference.ml.azure.com/swagger.json', 'name': 'pima-endpoint-sdkv2-v5', 'description': 'this is a sample online endpoint for mlflow model', 'tags': {'SDK': '2'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/resourcegroups/us_azure_practice/providers/microsoft.machinelearningservices/workspaces/us_azure/onlineendpoints/pima-endpoint-sdkv2-v5', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/providers/Microsoft.MachineLearningServices/locations/centralindia/mfeOperationsStatus/oe:a78ffdcd-763f-4353-b2d5-5456db1b024f:8fef899b-7447-4f01-a29d-531360081441?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/ba5d6a04-

##### Step 13: Remove old deployment

In [20]:
ml_client.online_deployments.begin_delete(name=deployment_name, endpoint_name=online_endpoint_name).wait()

##### Step 14: Delete Endpoint


In [21]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)

..................................................................